In [2]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service

from selenium import webdriver

import random

import numpy as np

import re

from selenium.webdriver.common.by import By

from selenium.webdriver.common.action_chains import ActionChains

In [3]:
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(
    options = my_options,
    service = my_service
)
# 開啟踩地雷遊戲
driver.get('https://minesweeper.online/tw/game/2115519617')

In [ ]:
list_elts = driver.find_elements(By.CSS_SELECTOR,'div[id^="cell"]')
width = int(list_elts[-1].get_attribute('data-x')) + 1
height = int(list_elts[-1].get_attribute('data-y')) + 1
np_elts = np.array([i.get_attribute('class') for i in list_elts]).reshape(height,width)
npblank_elts = np_elts == 'cell size24 hd_closed'
# 取的字樣1的位置 用陣列表示 字樣1為True 不是字樣1為False
np1_elts = np_elts == 'cell size24 hd_opened hd_type1'
# 取的字樣2的位置 用陣列表示 字樣2為True 不是字樣2為False
np2_elts = np_elts == 'cell size24 hd_opened hd_type2'
# 取的字樣3的位置 用陣列表示 字樣3為True 不是字樣3為False
np3_elts = np_elts == 'cell size24 hd_opened hd_type3'
# 取的字樣4的位置 用陣列表示 字樣4為True 不是字樣4為False
np4_elts = np_elts == 'cell size24 hd_opened hd_type4'
# 取的旗子的位置 用陣列表示 取的旗子為True 不是旗子為False
npflag_elts = np_elts == 'cell size24 hd_closed hd_flag'

In [4]:
def check_mine(number:int,np_input_elts:np.ndarray) :
    '''
        邊界檢查
        判定地雷位置
        更新陣列
        插旗子
    '''
    np_final_nine_elts = np.full(np_elts.shape,False,dtype = bool)
    np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
    for i in np.argwhere(np_input_elts):
        # np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        if i[0] == 0 and i[1] == 0 :
            np_nine_elts[0][1] = True
            np_nine_elts[1][0] = True
            np_nine_elts[1][1] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                # 九宮格mask和空格mask的交集為地雷位置 , 再將地雷位置放進np_final_nine_elts標記True
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                # 重製九宮格bool
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == 0 and i[1] == width - 1 :
            np_nine_elts[0][width - 2] = True
            np_nine_elts[1][width - 2] = True
            np_nine_elts[1][width - 1] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == height - 1  and i[1] == width - 1 :
            np_nine_elts[height - 1][width - 2] = True
            np_nine_elts[height - 2][width - 2] = True
            np_nine_elts[height - 2][width - 1] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == height - 1 and i[1] == 0:
            np_nine_elts[height - 1][1] = True
            np_nine_elts[height - 2][1] = True
            np_nine_elts[height - 2][0] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == 0 :
            np_nine_elts[0][i[1] - 1] = True
            np_nine_elts[0][i[1] + 1] = True
            np_nine_elts[1][i[1] - 1] = True
            np_nine_elts[1][i[1] + 1] = True
            np_nine_elts[1][i[1]]  = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == height - 1:
            np_nine_elts[height - 1][i[1] - 1] = True
            np_nine_elts[height - 1][i[1] + 1] = True
            np_nine_elts[height - 2][i[1] - 1] = True
            np_nine_elts[height - 2][i[1] + 1] = True
            np_nine_elts[height - 2][i[1]]  = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[1] == 0 :
            np_nine_elts[i[0] - 1][0] = True
            np_nine_elts[i[0] + 1][0] = True
            np_nine_elts[i[0] - 1][1] = True
            np_nine_elts[i[0] + 1][1] = True
            np_nine_elts[i[0]][1] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[1] == width - 1:
            np_nine_elts[i[0] - 1][width - 2] = True
            np_nine_elts[i[0] + 1][width - 2] = True
            np_nine_elts[i[0] - 1][width - 1] = True
            np_nine_elts[i[0] + 1][width - 1] = True
            np_nine_elts[i[0]][width - 2] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        else:
            np_nine_elts[i[0] - 1][i[1] - 1] = True
            np_nine_elts[i[0] - 1][i[1]] = True
            np_nine_elts[i[0] - 1][i[1] + 1] = True
            np_nine_elts[i[0]][i[1] - 1] = True
            np_nine_elts[i[0]][i[1] + 1] = True
            np_nine_elts[i[0] + 1][i[1] - 1] = True
            np_nine_elts[i[0] + 1][i[1]] = True
            np_nine_elts[i[0] + 1][i[1] + 1] = True
            if np.sum((np_nine_elts&(npflag_elts|npblank_elts))) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)           
    # 插旗子
    #  np_final_nine_elts 用argwhere得到每個地雷索引 在用[:,0] 切成row的一維陣列 放在np_final_row裡
    if np.sum(np_final_nine_elts) > 0 :
        np_final_row = np.argwhere(np_final_nine_elts)[:,0]
        #  np_final_nine_elts 用argwhere得到每個地雷索引 在用[:,1] 切成column的一維陣列 放在np_final_column裡
        np_final_column = np.argwhere(np_final_nine_elts)[:,1]
        # npblank_elts 地雷區原本空格的地方標記成False
        npblank_elts[np_final_row,np_final_column] = False
        # npblank_elts 地雷區插上旗子標記成True
        npflag_elts[np_final_row,np_final_column] = True
        # 將row與column用字串拼接好，呼叫該element(地雷區)，再將element點擊右鍵 np_final_column[index] =>column , obj => row
        for index,obj in enumerate(np_final_row) :
            # 格子的id名稱 => div#cell_x_y
            clickblank = driver.find_element(By.CSS_SELECTOR, f'div#cell_{np_final_column[index]}_{obj}')
            ac = ActionChains(driver)
            ac.context_click(clickblank)
            ac.perform()
    else :
        return

In [6]:
def check_blank(number:int , np_input_elts:np.ndarray) -> bool:
    '''
        邊界檢查
        判定優先點選空格
        更新陣列
        點空格
    '''
    global randomflag
    np_final_nine_elts = np.full(np_elts.shape,False,dtype = bool)
    np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
    for i in np.argwhere(np_input_elts):
        np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        if i[0] == 0 and i[1] == 0 :
            np_nine_elts[0][1] = True
            np_nine_elts[1][0] = True
            np_nine_elts[1][1] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == 0 and i[1] == width - 1 :
            np_nine_elts[0][width - 2] = True
            np_nine_elts[1][width - 2] = True
            np_nine_elts[1][width - 1] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == height - 1  and i[1] == width - 1 :
            np_nine_elts[height - 1][width - 2] = True
            np_nine_elts[height - 2][width - 2] = True
            np_nine_elts[height - 2][width - 1] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == height - 1 and i[1] == 0:
            np_nine_elts[height - 1][1] = True
            np_nine_elts[height - 2][1] = True
            np_nine_elts[height - 2][0] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == 0 :
            np_nine_elts[0][i[1] - 1] = True
            np_nine_elts[0][i[1] + 1] = True
            np_nine_elts[1][i[1] - 1] = True
            np_nine_elts[1][i[1] + 1] = True
            np_nine_elts[1][i[1]]  = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[0] == height - 1:
            np_nine_elts[height - 1][i[1] - 1] = True
            np_nine_elts[height - 1][i[1] + 1] = True
            np_nine_elts[height - 2][i[1] - 1] = True
            np_nine_elts[height - 2][i[1] + 1] = True
            np_nine_elts[height - 2][i[1]]  = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[1] == 0 :
            np_nine_elts[i[0] - 1][0] = True
            np_nine_elts[i[0] + 1][0] = True
            np_nine_elts[i[0] - 1][1] = True
            np_nine_elts[i[0] + 1][1] = True
            np_nine_elts[i[0]][1] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        elif i[1] == width - 1:
            np_nine_elts[i[0] - 1][width - 2] = True
            np_nine_elts[i[0] + 1][width - 2] = True
            np_nine_elts[i[0] - 1][width - 1] = True
            np_nine_elts[i[0] + 1][width - 1] = True
            np_nine_elts[i[0]][width - 2] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
        else:
            np_nine_elts[i[0] - 1][i[1] - 1] = True
            np_nine_elts[i[0] - 1][i[1]] = True
            np_nine_elts[i[0] - 1][i[1] + 1] = True
            np_nine_elts[i[0]][i[1] - 1] = True
            np_nine_elts[i[0]][i[1] + 1] = True
            np_nine_elts[i[0] + 1][i[1] - 1] = True
            np_nine_elts[i[0] + 1][i[1]] = True
            np_nine_elts[i[0] + 1][i[1] + 1] = True
            if np.sum((np_nine_elts&npflag_elts)) == number:
                np_final_nine_elts[np_nine_elts&npblank_elts] = True
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
            else :
                np_nine_elts = np.full(np_elts.shape,False,dtype = bool)
    if np.sum(np_final_nine_elts) > 0 :
        randomflag = False
        np_final_row = np.argwhere(np_final_nine_elts)[:,0]
        np_final_column = np.argwhere(np_final_nine_elts)[:,1]
        npblank_elts[np_final_row,np_final_column] = False
        for index,obj in enumerate(np_final_row) :
            clickblank = driver.find_element(By.CSS_SELECTOR, f'div#cell_{np_final_column[index]}_{obj}')
            ac = ActionChains(driver)
            ac.click(clickblank)
            ac.perform()
    else:
        return

In [5]:
def random_pick(randomflag:bool):
    if np.sum(npblank_elts) > 0 and randomflag == True:
        rd_number = random.randint(0,np.sum(npblank_elts) - 1)
        rd_column = np.argwhere(npblank_elts)[rd_number][1]
        rd_row = np.argwhere(npblank_elts)[rd_number][0]
        clickbuttton = driver.find_element(By.CSS_SELECTOR,  f'div#cell_{rd_column}_{rd_row}')
        clickbuttton.click()

In [12]:
# main區

restartElement = driver.find_element(By.CSS_SELECTOR, 'div#top_area_face')
restartElement.click()
list_elts = driver.find_elements(By.CSS_SELECTOR,'div[id^="cell"]')
width = int(list_elts[-1].get_attribute('data-x')) + 1
height = int(list_elts[-1].get_attribute('data-y')) + 1
while True :
    if 'top-area-face zoomable hd_top-area-face-lose' == restartElement.get_attribute('class'):
        break
    elif 'top-area-face zoomable hd_top-area-face-win' == restartElement.get_attribute('class'):
        break
    randomflag = True
    list_elts = driver.find_elements(By.CSS_SELECTOR,'div[id^="cell"]') 
    np_elts = np.array([i.get_attribute('class') for i in list_elts]).reshape(height,width)
    npblank_elts = np_elts == 'cell size24 hd_closed'
    # 取的字樣1的位置 用陣列表示 字樣1為True 不是字樣1為False
    np1_elts = np_elts == 'cell size24 hd_opened hd_type1'
    # 取的字樣2的位置 用陣列表示 字樣2為True 不是字樣2為False
    np2_elts = np_elts == 'cell size24 hd_opened hd_type2'
    # 取的字樣3的位置 用陣列表示 字樣3為True 不是字樣3為False
    np3_elts = np_elts == 'cell size24 hd_opened hd_type3'
    # 取的字樣4的位置 用陣列表示 字樣4為True 不是字樣4為False
    np4_elts = np_elts == 'cell size24 hd_opened hd_type4'
    # 取的旗子的位置 用陣列表示 取的旗子為True 不是旗子為False
    npflag_elts = np_elts == 'cell size24 hd_closed hd_flag'
    
    check_mine(4,np4_elts)
    check_mine(3,np3_elts)
    check_mine(2,np2_elts)
    check_mine(1,np1_elts)
    check_blank(4,np4_elts)
    check_blank(3,np3_elts)
    check_blank(2,np2_elts)
    check_blank(1,np1_elts)
    random_pick(randomflag)
    